In [1]:
import pandas as pd
import sqlite3
import numpy as np

In [2]:
#importing all the required ML packages
from sklearn.linear_model import LogisticRegression #logistic regression
from sklearn import svm #support vector Machine
from sklearn.ensemble import RandomForestClassifier #Random Forest
from sklearn.neighbors import KNeighborsClassifier #KNN
from sklearn.naive_bayes import GaussianNB #Naive bayes
from sklearn.tree import DecisionTreeClassifier #Decision Tree
from sklearn.model_selection import train_test_split #training and testing data split
from sklearn import metrics #accuracy measure
from sklearn.metrics import confusion_matrix #for confusion matrix

# Generate and plot a synthetic imbalanced classification dataset
from collections import Counter
from sklearn.datasets import make_classification
from matplotlib import pyplot


### Importación de tablas

In [3]:
conn = sqlite3.connect('database.sqlite')

In [4]:
team_query = '''select * from Team''' 
player_query = '''select * from Player'''
player_attributes_query = '''select * from Player_Attributes'''
match_query = '''select * from Match'''
league_query = '''select * from League'''

In [5]:
team = pd.read_sql_query(team_query,conn)
player = pd.read_sql_query(player_query,conn)
player_attributes = pd.read_sql_query(player_attributes_query,conn)
match = pd.read_sql_query(match_query,conn)
league = pd.read_sql_query(league_query,conn)

### Exploración base de datos, selección de principales variables y primer preprocesamiento

In [6]:
league.head()

,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League
2,4769,4769,France Ligue 1
3,7809,7809,Germany 1. Bundesliga
4,10257,10257,Italy Serie A


In [7]:
match.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


In [8]:
team.head(10)

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB
5,6,8635,229.0,RSC Anderlecht,AND
6,7,9991,674.0,KAA Gent,GEN
7,8,9998,1747.0,RAEC Mons,MON
8,9,7947,NaN,FCV Dender EH,DEN
9,10,9985,232.0,Standard de Liège,STL


In [9]:
# Populating Team Names

match['home_team'] = match['home_team_api_id'].map(team.set_index('team_api_id')['team_long_name'])
match['away_team'] = match['away_team_api_id'].map(team.set_index('team_api_id')['team_long_name'])
match['league_country'] = match['country_id'].map(league.set_index('country_id')['name'])

In [10]:
status_values = ['win', 'lose', 'draw']

general_values = ['home win', 'away win', 'draw']
general_encoded_values = [0, 1, 2]

home_conditions = [(match['home_team_goal']>match['away_team_goal']), (match['home_team_goal']<match['away_team_goal']), (match['home_team_goal']==match['away_team_goal'])]

away_conditions = [(match['home_team_goal']<match['away_team_goal']), (match['home_team_goal']>match['away_team_goal']), (match['home_team_goal']==match['away_team_goal'])]

In [11]:
match['general_result'] = np.select(home_conditions, general_values)

In [12]:
match['general_encoded_result'] = np.select(home_conditions, general_encoded_values)

In [13]:
match['home_result'] = np.select(home_conditions, status_values)

In [14]:
match['away_result'] = np.select(away_conditions, status_values)

In [15]:
# Selecting main columns:

match = match[['date', 'id', 'league_id', 'country_id', 'league_country', 'season', 'match_api_id', 'home_team_api_id', 'home_team', 'away_team_api_id', 'away_team', 'home_team_goal', 'away_team_goal', 'home_result', 'general_result', 'general_encoded_result', 'away_result', 'B365H', 'B365A', 'B365D']]

In [16]:
match.head(10)

,date,id,league_id,country_id,league_country,season,match_api_id,home_team_api_id,home_team,away_team_api_id,away_team,home_team_goal,away_team_goal,home_result,general_result,general_encoded_result,away_result,B365H,B365A,B365D
0,2008-08-17 00:00:00,1,1,1,Belgium Jupiler League,2008/2009,492473,9987,KRC Genk,9993,Beerschot AC,1,1,draw,draw,2,draw,1.73,5.00,3.40
1,2008-08-16 00:00:00,2,1,1,Belgium Jupiler League,2008/2009,492474,10000,SV Zulte-Waregem,9994,Sporting Lokeren,0,0,draw,draw,2,draw,1.95,3.60,3.20
2,2008-08-16 00:00:00,3,1,1,Belgium Jupiler League,2008/2009,492475,9984,KSV Cercle Brugge,8635,RSC Anderlecht,0,3,lose,away win,1,win,2.38,2.75,3.30
3,2008-08-17 00:00:00,4,1,1,Belgium Jupiler League,2008/2009,492476,9991,KAA Gent,9998,RAEC Mons,5,0,win,home win,0,lose,1.44,7.50,3.75
4,2008-08-16 00:00:00,5,1,1,Belgium Jupiler League,2008/2009,492477,7947,FCV Dender EH,9985,Standard de Liège,1,3,lose,away win,1,win,5.00,1.65,3.50
5,2008-09-24 00:00:00,6,1,1,Belgium Jupiler League,2008/2009,492478,8203,KV Mechelen,8342,Club Brugge KV,1,1,draw,draw,2,draw,4.75,1.67,3.40
6,2008-08-16 00:00:00,7,1,1,Belgium Jupiler League,2008/2009,492479,9999,KSV Roeselare,8571,KV Kortrijk,2,2,draw,draw,2,draw,2.10,3.30,3.20
7,2008-08-16 00:00:00,8,1,1,Belgium Jupiler League,2008/2009,492480,4049,Tubize,9996,Royal Excel Mouscron,1,2,lose,away win,1,win,3.20,2.20,3.40
8,2008-08-16 00:00:00,9,1,1,Belgium Jupiler League,2008/2009,492481,10001,KVC Westerlo,9986,Sporting Charleroi,1,0,win,home win,0,lose,2.25,2.88,3.25
9,2008-11-01 00:00:00,10,1,1,Belgium Jupiler League,2008/2009,492564,8342,Club Brugge KV,8571,KV Kortrijk,4,1,win,home win,0,lose,1.30,9.50,5.25


In [17]:
match.league_country.value_counts()

Spain LIGA BBVA             3040
France Ligue 1              3040
England Premier League      3040
Italy Serie A               3017
Germany 1. Bundesliga       2448
Netherlands Eredivisie      2448
Portugal Liga ZON Sagres    2052
Poland Ekstraklasa          1920
Scotland Premier League     1824
Belgium Jupiler League      1728
Switzerland Super League    1422
Name: league_country, dtype: int64

In [18]:
# Amount of NA's

match.isna().sum()

date                         0
id                           0
league_id                    0
country_id                   0
league_country               0
season                       0
match_api_id                 0
home_team_api_id             0
home_team                    0
away_team_api_id             0
away_team                    0
home_team_goal               0
away_team_goal               0
home_result                  0
general_result               0
general_encoded_result       0
away_result                  0
B365H                     3387
B365A                     3387
B365D                     3387
dtype: int64

In [19]:
# Countries with missing values on Betting columns information --> Remove Poland and Switzerland and fill other countrie's NA's with mean

match[match['B365H'].isna()].league_country.value_counts()

Poland Ekstraklasa          1920
Switzerland Super League    1422
Belgium Jupiler League        22
Portugal Liga ZON Sagres       8
Italy Serie A                  6
France Ligue 1                 4
Netherlands Eredivisie         3
Germany 1. Bundesliga          1
Spain LIGA BBVA                1
Name: league_country, dtype: int64

In [20]:
# Removing Poland and Switzerland

match_final = match.loc[(match['league_country'] != 'Poland Ekstraklasa') & (match['league_country'] != 'Switzerland Super League')]

In [21]:
match_final.groupby('league_country').mean()

,id,league_id,country_id,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,general_encoded_result,B365H,B365A,B365D
league_country,,,,,,,,,,,,
Belgium Jupiler League,864.5,1.0,1.0,1.123332e+06,13810.710648,13810.710648,1.609375,1.192130,0.777199,2.412849,4.310481,3.751553
England Premier League,3248.5,1729.0,1729.0,1.178767e+06,9142.275000,9142.275000,1.550987,1.159539,0.800329,2.701964,4.910437,3.952720
France Ligue 1,6288.5,4769.0,4769.0,1.171642e+06,9808.300000,9808.300000,1.402961,1.040132,0.835526,2.409911,4.389489,3.437790
Germany 1. Bundesliga,9032.5,7809.0,7809.0,1.189377e+06,9180.250000,9180.250000,1.626634,1.274918,0.791667,2.596530,4.375509,3.913788
Italy Serie A,11765.0,10257.0,10257.0,1.236752e+06,10345.651641,10346.633079,1.500829,1.116009,0.797481,2.506616,4.555254,3.599661
Netherlands Eredivisie,14497.5,13274.0,13274.0,1.180382e+06,9181.388889,9181.388889,1.779820,1.301062,0.758987,2.590569,4.916953,4.124834
Portugal Liga ZON Sagres,18667.5,17642.0,17642.0,1.246115e+06,13952.262671,13952.262671,1.408382,1.126218,0.817251,2.938415,4.784946,3.804976
Scotland Premier League,20605.5,19694.0,19694.0,1.204328e+06,9130.754386,9130.599781,1.429276,1.204496,0.828399,2.800493,4.174359,3.821765
Spain LIGA BBVA,23037.5,21518.0,21518.0,1.214688e+06,9032.287500,9032.287500,1.631250,1.135855,0.743092,2.762152,5.226018,4.160408


In [22]:
### Filling NA's with mean -->> See how to automate with a function

match_final.loc[(match_final.league_country == 'Belgium Jupiler League') & (match_final.B365D.isnull()), 'B365D'] = 3.751
match_final.loc[(match_final.league_country == 'Portugal Liga ZON Sagres') & (match_final.B365D.isnull()), 'B365D'] = 3.804
match_final.loc[(match_final.league_country == 'Italy Serie A') & (match_final.B365D.isnull()), 'B365D'] = 3.599
match_final.loc[(match_final.league_country == 'France Ligue 1') & (match_final.B365D.isnull()), 'B365D'] = 3.437
match_final.loc[(match_final.league_country == 'Netherlands Eredivisie') & (match_final.B365D.isnull()), 'B365D'] = 4.124
match_final.loc[(match_final.league_country == 'Spain LIGA BBVA') & (match_final.B365D.isnull()), 'B365D'] = 4.16
match_final.loc[(match_final.league_country == 'Germany 1. Bundesliga') & (match_final.B365D.isnull()), 'B365D'] = 3.913

match_final.loc[(match_final.league_country == 'Belgium Jupiler League') & (match_final.B365A.isnull()), 'B365A'] = 4.310
match_final.loc[(match_final.league_country == 'Portugal Liga ZON Sagres') & (match_final.B365A.isnull()), 'B365A'] = 4.784
match_final.loc[(match_final.league_country == 'Italy Serie A') & (match_final.B365A.isnull()), 'B365A'] = 4.555
match_final.loc[(match_final.league_country == 'France Ligue 1') & (match_final.B365A.isnull()), 'B365A'] = 4.389
match_final.loc[(match_final.league_country == 'Netherlands Eredivisie') & (match_final.B365A.isnull()), 'B365A'] = 4.916
match_final.loc[(match_final.league_country == 'Spain LIGA BBVA') & (match_final.B365A.isnull()), 'B365A'] = 5.22
match_final.loc[(match_final.league_country == 'Germany 1. Bundesliga') & (match_final.B365A.isnull()), 'B365A'] = 4.375

match_final.loc[(match_final.league_country == 'Belgium Jupiler League') & (match_final.B365H.isnull()), 'B365H'] = 2.412
match_final.loc[(match_final.league_country == 'Portugal Liga ZON Sagres') & (match_final.B365H.isnull()), 'B365H'] = 2.938
match_final.loc[(match_final.league_country == 'Italy Serie A') & (match_final.B365H.isnull()), 'B365H'] = 2.506
match_final.loc[(match_final.league_country == 'France Ligue 1') & (match_final.B365H.isnull()), 'B365H'] = 2.409
match_final.loc[(match_final.league_country == 'Netherlands Eredivisie') & (match_final.B365H.isnull()), 'B365H'] = 2.590
match_final.loc[(match_final.league_country == 'Spain LIGA BBVA') & (match_final.B365H.isnull()), 'B365H'] = 2.762
match_final.loc[(match_final.league_country == 'Germany 1. Bundesliga') & (match_final.B365H.isnull()), 'B365H'] = 2.596

C:\Users\tomi_\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [23]:
match_final['3+_goals_diff'] = np.where(((match_final['home_team_goal']-match_final['away_team_goal']>2) | (match_final['away_team_goal']-match_final['home_team_goal']>2)), 1, 0)

C:\Users\tomi_\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [24]:
match_final.head()

,date,id,league_id,country_id,league_country,season,match_api_id,home_team_api_id,home_team,away_team_api_id,...,home_team_goal,away_team_goal,home_result,general_result,general_encoded_result,away_result,B365H,B365A,B365D,3+_goals_diff
0,2008-08-17 00:00:00,1,1,1,Belgium Jupiler League,2008/2009,492473,9987,KRC Genk,9993,...,1,1,draw,draw,2,draw,1.73,5.00,3.40,0
1,2008-08-16 00:00:00,2,1,1,Belgium Jupiler League,2008/2009,492474,10000,SV Zulte-Waregem,9994,...,0,0,draw,draw,2,draw,1.95,3.60,3.20,0
2,2008-08-16 00:00:00,3,1,1,Belgium Jupiler League,2008/2009,492475,9984,KSV Cercle Brugge,8635,...,0,3,lose,away win,1,win,2.38,2.75,3.30,1
3,2008-08-17 00:00:00,4,1,1,Belgium Jupiler League,2008/2009,492476,9991,KAA Gent,9998,...,5,0,win,home win,0,lose,1.44,7.50,3.75,1
4,2008-08-16 00:00:00,5,1,1,Belgium Jupiler League,2008/2009,492477,7947,FCV Dender EH,9985,...,1,3,lose,away win,1,win,5.00,1.65,3.50,0


In [25]:
### Defining X and y

X = match_final.drop(['general_result', 'general_encoded_result'], axis=1)
y = match_final['general_result']


In [26]:
match_final.tail()

,date,id,league_id,country_id,league_country,season,match_api_id,home_team_api_id,home_team,away_team_api_id,...,home_team_goal,away_team_goal,home_result,general_result,general_encoded_result,away_result,B365H,B365A,B365D,3+_goals_diff
24552,2015-10-25 00:00:00,24553,21518,21518,Spain LIGA BBVA,2015/2016,2030167,9906,Atlético Madrid,10267,...,2,1,win,home win,0,lose,1.57,6.50,3.80,0
24553,2015-10-24 00:00:00,24554,21518,21518,Spain LIGA BBVA,2015/2016,2030168,9864,Málaga CF,9783,...,2,0,win,home win,0,lose,2.25,3.40,3.25,0
24554,2015-10-26 00:00:00,24555,21518,21518,Spain LIGA BBVA,2015/2016,2030169,8315,Athletic Club de Bilbao,9869,...,3,0,win,home win,0,lose,1.53,7.00,4.00,1
24555,2015-10-24 00:00:00,24556,21518,21518,Spain LIGA BBVA,2015/2016,2030170,7878,Granada CF,8603,...,1,1,draw,draw,2,draw,2.30,3.25,3.25,0
24556,2015-10-23 00:00:00,24557,21518,21518,Spain LIGA BBVA,2015/2016,2030171,8370,Rayo Vallecano,8558,...,3,0,win,home win,0,lose,2.20,3.20,3.40,1


# EJERCICIO 1

## Simple Random Sample

Using a third of the population as sample

In [27]:
sample_size = round((len(match_final)/3), 0)
#sample_size = int(match_final)
sample_size

7546.0

In [28]:
random_sample = match_final.sample(n=int(sample_size), random_state=42)
random_sample

,date,id,league_id,country_id,league_country,season,match_api_id,home_team_api_id,home_team,away_team_api_id,...,home_team_goal,away_team_goal,home_result,general_result,general_encoded_result,away_result,B365H,B365A,B365D,3+_goals_diff
4579,2016-04-20 00:00:00,4580,1729,1729,England Premier League,2015/2016,1988971,8650,Liverpool,8668,...,4,0,win,home win,0,lose,1.73,5.00,4.00,1
12893,2015-08-23 00:00:00,12894,10257,10257,Italy Serie A,2015/2016,2060255,8534,Empoli,8533,...,1,3,lose,away win,1,win,2.15,3.60,3.25,0
134,2009-02-14 00:00:00,135,1,1,Belgium Jupiler League,2008/2009,493006,9996,Royal Excel Mouscron,10001,...,0,0,draw,draw,2,draw,2.40,2.70,3.30,0
14887,2013-12-22 00:00:00,14888,13274,13274,Netherlands Eredivisie,2013/2014,1473179,8674,FC Groningen,8464,...,5,2,win,home win,0,lose,1.67,4.75,4.00,1
7159,2014-08-15 00:00:00,7160,4769,4769,France Ligue 1,2014/2015,1709708,7819,SM Caen,8639,...,0,1,lose,away win,1,win,3.60,2.15,3.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2807,2011-05-22 00:00:00,2808,1729,1729,England Premier League,2010/2011,840234,10194,Stoke City,8528,...,0,1,lose,away win,1,win,2.75,2.40,3.60,0
18740,2013-04-05 00:00:00,18741,17642,17642,Portugal Liga ZON Sagres,2012/2013,1256537,10212,CS Marítimo,6403,...,1,1,draw,draw,2,draw,2.38,3.20,3.00,0
19175,2015-01-31 00:00:00,19176,17642,17642,Portugal Liga ZON Sagres,2014/2015,1750641,6547,FC Penafiel,7844,...,1,1,draw,draw,2,draw,4.50,1.73,3.75,0
19219,2015-02-27 00:00:00,19220,17642,17642,Portugal Liga ZON Sagres,2014/2015,1750676,7844,Vitória Guimarães,10212,...,1,0,win,home win,0,lose,1.75,4.33,3.75,0


In [29]:
# Sample size verification

len(random_sample)

7546

### Simple random sampling using train test split

In [30]:
X_train_random, X_test_random, y_train_random, y_test_random = train_test_split(X, y, test_size = 1/3, random_state = 42) 

In [31]:
y_train_random.describe()

count        15091
unique           3
top       home win
freq          6921
Name: general_result, dtype: object

In [32]:
y_test_random.describe()

count         7546
unique           3
top       home win
freq          3476
Name: general_result, dtype: object

## Systematic Sample

In [33]:
# Define systematic sampling function
def systematic_sampling(match_final, step):
  
    indexes = np.arange(0, len(match_final), step=step)
    systematic_sample = match_final.iloc[indexes]
    return systematic_sample
  
  
# Obtain a systematic sample and save it in a new variable
systematic_sample = systematic_sampling(match_final, 3)
  
# View sampled data frame
print('Original sample has been divided into interval of 3 resulting in a new sample of', len(systematic_sample), 'records.')

Original sample has been divided into interval of 3 resulting in a new sample of 7546 records.


# EJERCICIO 2

## Stratified Sample

In [34]:
#define total sample size desired
N = 7546

#perform stratified random sampling
stratified_sample = match_final.groupby('league_country', group_keys=False).apply(lambda x: x.sample(int(np.rint(N*len(x)/len(match_final))))).sample(frac=1).reset_index(drop=True)

In [35]:
stratified_sample.head()

,date,id,league_id,country_id,league_country,season,match_api_id,home_team_api_id,home_team,away_team_api_id,...,home_team_goal,away_team_goal,home_result,general_result,general_encoded_result,away_result,B365H,B365A,B365D,3+_goals_diff
0,2008-11-30 00:00:00,10308,10257,10257,Italy Serie A,2008/2009,539688,10233,Genoa,9857,...,1,1,draw,draw,2,draw,1.62,6.50,3.4,0
1,2009-02-01 00:00:00,10400,10257,10257,Italy Serie A,2008/2009,539772,10233,Genoa,8540,...,1,0,win,home win,0,lose,1.90,4.75,3.2,0
2,2012-04-11 00:00:00,11633,10257,10257,Italy Serie A,2011/2012,1083224,8636,Inter,8551,...,2,1,win,home win,0,lose,1.62,5.50,3.8,0
3,2014-03-07 00:00:00,18959,17642,17642,Portugal Liga ZON Sagres,2013/2014,1498185,7841,Rio Ave FC,10215,...,0,0,draw,draw,2,draw,1.95,4.20,3.2,0
4,2011-09-10 00:00:00,8992,7809,7809,Germany 1. Bundesliga,2011/2012,1029843,10269,VfB Stuttgart,9904,...,3,0,win,home win,0,lose,1.85,4.00,3.6,1


STRATIFIED SAMPLE VERIFICATION

In [36]:
stratified_sample.league_country.value_counts()/len(stratified_sample)

Spain LIGA BBVA             0.134261
England Premier League      0.134261
France Ligue 1              0.134261
Italy Serie A               0.133333
Germany 1. Bundesliga       0.108151
Netherlands Eredivisie      0.108151
Portugal Liga ZON Sagres    0.090656
Scotland Premier League     0.080583
Belgium Jupiler League      0.076342
Name: league_country, dtype: float64

In [37]:
match_final.league_country.value_counts()/len(match_final)

Spain LIGA BBVA             0.134293
France Ligue 1              0.134293
England Premier League      0.134293
Italy Serie A               0.133277
Germany 1. Bundesliga       0.108142
Netherlands Eredivisie      0.108142
Portugal Liga ZON Sagres    0.090648
Scotland Premier League     0.080576
Belgium Jupiler League      0.076335
Name: league_country, dtype: float64

### Stratified sample using train_test_split

In [38]:
X_train_stratified, X_test_stratified, y_train_stratified, y_test_stratified = train_test_split(X, y, test_size=1/3, stratify=y)

In [39]:
y_train_stratified.value_counts()/len(y_train_stratified)

home win    0.459280
away win    0.287787
draw        0.252932
Name: general_result, dtype: float64

In [40]:
y_test_stratified.value_counts()/len(y_test_stratified)

home win    0.459316
away win    0.287835
draw        0.252849
Name: general_result, dtype: float64

### SMOTE Sampling (synthetic minority oversample techniche)

In [41]:
#pip install imbalanced-learn --user

In [42]:
from imblearn.over_sampling import SMOTE 

In [43]:
# Identification of imbalanced proportion in sample: 

match_final['3+_goals_diff'].value_counts()/len(match_final)

0    0.843398
1    0.156602
Name: 3+_goals_diff, dtype: float64

In [44]:
# Selection of numerical values to implement SMOTE analysis

X_smote = match_final[['id', 'league_id', 'home_team_api_id', 'home_team_api_id', 'home_team_goal', 'away_team_goal', 'B365H', 'B365A', 'B365D', '3+_goals_diff']].values
y_smote = match_final['general_encoded_result']

In [45]:
sm = SMOTE(random_state=42)

X_sm, y_sm = sm.fit_resample(X_smote, y_smote)

print(f'''Shape of X before SMOTE: {X.shape}
Shape of X after SMOTE: {X_sm.shape}''')

print('\nBalance of positive and negative classes (%):')
y_sm.value_counts(normalize=True) * 100

Shape of X before SMOTE: (22637, 19)
Shape of X after SMOTE: (31191, 10)

Balance of positive and negative classes (%):


2    33.333333
1    33.333333
0    33.333333
Name: general_encoded_result, dtype: float64

# EJERCICIO 3

## Reservoir Sampling

In [46]:
import random

In [47]:
stream = match_final.to_numpy()

In [49]:
N=7546
reservoir = []

# Function definition

for i, element in enumerate(stream):
    if i+1<=N:
        reservoir.append(element)
    else:
        probability = N/(i+N)
        if random.random() < probability:
            reservoir[random.choice(range(0,N))] = element
            

print(reservoir)



[array(['2008-08-17 00:00:00', 1, 1, 1, 'Belgium Jupiler League',
       '2008/2009', 492473, 9987, 'KRC Genk', 9993, 'Beerschot AC', 1, 1,
       'draw', 'draw', 2, 'draw', 1.73, 5.0, 3.4, 0], dtype=object), array(['2013-09-21 00:00:00', 9613, 7809, 7809, 'Germany 1. Bundesliga',
       '2013/2014', 1479039, 8721, 'VfL Wolfsburg', 8226,
       'TSG 1899 Hoffenheim', 2, 1, 'win', 'home win', 0, 'lose', 1.73,
       4.2, 4.0, 0], dtype=object), array(['2008-08-16 00:00:00', 3, 1, 1, 'Belgium Jupiler League',
       '2008/2009', 492475, 9984, 'KSV Cercle Brugge', 8635,
       'RSC Anderlecht', 0, 3, 'lose', 'away win', 1, 'win', 2.38, 2.75,
       3.3, 1], dtype=object), array(['2016-03-04 00:00:00', 19549, 17642, 17642,
       'Portugal Liga ZON Sagres', '2015/2016', 2016033, 6403,
       'FC Paços de Ferreira', 10212, 'CS Marítimo', 2, 2, 'draw', 'draw',
       2, 'draw', 2.45, 2.9, 3.25, 0], dtype=object), array(['2008-08-16 00:00:00', 5, 1, 1, 'Belgium Jupiler League',
       '2008/2

In [50]:
reservoir_match_final = pd.DataFrame(data=reservoir, columns=match_final.columns)

In [51]:
reservoir_match_final

,date,id,league_id,country_id,league_country,season,match_api_id,home_team_api_id,home_team,away_team_api_id,...,home_team_goal,away_team_goal,home_result,general_result,general_encoded_result,away_result,B365H,B365A,B365D,3+_goals_diff
0,2008-08-17 00:00:00,1,1,1,Belgium Jupiler League,2008/2009,492473,9987,KRC Genk,9993,...,1,1,draw,draw,2,draw,1.73,5.00,3.40,0
1,2013-09-21 00:00:00,9613,7809,7809,Germany 1. Bundesliga,2013/2014,1479039,8721,VfL Wolfsburg,8226,...,2,1,win,home win,0,lose,1.73,4.20,4.00,0
2,2008-08-16 00:00:00,3,1,1,Belgium Jupiler League,2008/2009,492475,9984,KSV Cercle Brugge,8635,...,0,3,lose,away win,1,win,2.38,2.75,3.30,1
3,2016-03-04 00:00:00,19549,17642,17642,Portugal Liga ZON Sagres,2015/2016,2016033,6403,FC Paços de Ferreira,10212,...,2,2,draw,draw,2,draw,2.45,2.90,3.25,0
4,2008-08-16 00:00:00,5,1,1,Belgium Jupiler League,2008/2009,492477,7947,FCV Dender EH,9985,...,1,3,lose,away win,1,win,5.00,1.65,3.50,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7541,2015-08-16 00:00:00,7542,4769,4769,France Ligue 1,2015/2016,1989791,8689,FC Lorient,7794,...,1,1,draw,draw,2,draw,1.85,4.75,3.40,0
7542,2015-08-14 00:00:00,7543,4769,4769,France Ligue 1,2015/2016,1989792,9829,AS Monaco,8639,...,0,0,draw,draw,2,draw,1.62,6.50,3.60,0
7543,2012-08-25 00:00:00,9040,7809,7809,Germany 1. Bundesliga,2012/2013,1239465,8357,SpVgg Greuther Fürth,9823,...,0,3,lose,away win,1,win,8.50,1.36,4.75,1
7544,2014-11-08 00:00:00,9667,7809,7809,Germany 1. Bundesliga,2014/2015,1732815,8226,TSG 1899 Hoffenheim,8722,...,3,4,lose,away win,1,win,1.80,4.00,3.80,0
